In [21]:
!pip install librosa

import librosa
import os
import json

In [22]:

#JSON_PATH = "data.json"
SAMPLES_TO_CONSIDER = 22050*1
SAMPLES_TO_CONSIDER

22050

In [23]:
def preprocess_dataset(dataset_path, json_path, num_mfcc=13, n_fft=2048, hop_length=512):

    data = {
        "mapping": [],
        "labels": [],
        "MFCCs": [],
        "files": []
    }
    count=0

    # loop through all sub-dirs
    for i, (dirpath, dirnames, filenames) in enumerate(os.walk(dataset_path)):

        # ensure we're at sub-folder level
        if dirpath is not dataset_path:

            # save label (i.e., sub-folder name) in the mapping
            label = dirpath.split("/")[-1]
            data["mapping"].append(label)
            print("\nProcessing: '{}'".format(label))

            # process all audio files in sub-dir and store MFCCs
            for f in filenames:
                file_path = os.path.join(dirpath, f)

                # load audio file and slice it to ensure length consistency among different files
                signal, sample_rate = librosa.load(file_path)

                # drop audio files with less than pre-decided number of samples
                if len(signal) >= SAMPLES_TO_CONSIDER:

                    # ensure consistency of the length of the signal
                    signal = signal[:SAMPLES_TO_CONSIDER]

                    # extract MFCCs
                    MFCCs = librosa.feature.mfcc(signal, sample_rate, n_mfcc=num_mfcc, n_fft=n_fft,
                                                    hop_length=hop_length)

                    # store data for analysed track
                    count=count+1
                    data["MFCCs"].append(MFCCs.T.tolist())
                    data["labels"].append(i-1)
                    data["files"].append(file_path)
                    print(str(count)+" {}: {}".format(file_path, i-1))

    # save data in json file
    with open(json_path, "w") as fp:
        json.dump(data, fp, indent=4)


In [24]:
preprocess_dataset('dataset/recordings/', 'newdata.json')


Processing: 'domestic'
1 dataset/recordings/domestic\audio 101.ogg: 0
2 dataset/recordings/domestic\audio 103.ogg: 0
3 dataset/recordings/domestic\audio 104.ogg: 0
4 dataset/recordings/domestic\audio 105.ogg: 0
5 dataset/recordings/domestic\audio 106.ogg: 0
6 dataset/recordings/domestic\audio 107.ogg: 0
7 dataset/recordings/domestic\audio 109.ogg: 0
8 dataset/recordings/domestic\audio 110.ogg: 0
9 dataset/recordings/domestic\audio 111.ogg: 0
10 dataset/recordings/domestic\audio 112.ogg: 0
11 dataset/recordings/domestic\audio 118.ogg: 0
12 dataset/recordings/domestic\audio 120.ogg: 0
13 dataset/recordings/domestic\audio 19.ogg: 0
14 dataset/recordings/domestic\audio 21.ogg: 0
15 dataset/recordings/domestic\audio 23.ogg: 0
16 dataset/recordings/domestic\audio 24.ogg: 0
17 dataset/recordings/domestic\audio 25.ogg: 0
18 dataset/recordings/domestic\audio 26.ogg: 0
19 dataset/recordings/domestic\audio 27.ogg: 0
20 dataset/recordings/domestic\audio 29.ogg: 0
21 dataset/recordings/domestic\au

In [25]:
#done